# Quantum Code (Charlie)

<span style="font-size:1.2em;">*Notes:*</span>

Bottleneck in the speed of this code seems to be solving the DE. May be able to speed up the construction of the superoperator by using multi-threading (https://docs.julialang.org/en/v1/manual/multi-threading/). Any feedback on how we can optimise this would be appreciated!

<span style="font-size:1.2em;">! Run this in **Julia** !</span> 

1. You'll need to install the Julia add on for Jupyter Notebooks (IJulia). Follow guide here: https://datatofish.com/add-julia-to-jupyter/
2. Go to *Kernel* -> *Change Kernel* -> *Julia*

## Packages

In [3]:
include("RCLib.jl") # Include the RC library with all the useful functions
using .RCLib

using LinearAlgebra # For IIL matrix multiplication
using OrdinaryDiffEq
using DifferentialEquations

## Plotting Packages ##
using CSV # Allows data to be converted to .CSV to use with Python for plotting
using DataFrames # Another useful package for exporting data

## Performance Packages ##
using BenchmarkTools

## Variables

### Lorentzian Spectral Density

In [4]:
prm = [1.4 0.001 1];

In [5]:
ω0, Γ, α = prm*ωL;

### RC-Specific

In [6]:
Ω = ω0;
λ = ωL*sqrt(α/Ω);
δ = Γ/(2π*Ω);
n = 5;

In [7]:
Htot(n) = HRC1D(n, λ, Ω);

### Environment

In [8]:
T = 1;

### Timescales

In [9]:
ti, tf, dt = 0.0, 20000.0, 100.0;
tspan = (ti, tf);
t = ti:dt:tf;

## Initial States

In [10]:
θ, ϕ = -π/2, 0; # Define some θ and ϕ on the Bloch sphere

In [12]:
state0 = ρ0(θ, ϕ, Ω, n, T);

## Solving the Master Equation

In [13]:
superop = 𝒮(n, Ω, λ, δ, T); # This is the superoperator corresponding to the Redfield master equation

In [14]:
alg = Vern7();

In [15]:
dstate(dρ, ρ, v, t) = mul!(dρ, superop, ρ); # Solves the DE
prob = ODEProblem(dstate, vec(state0), tspan);
sol = solve(prob, alg);
solF(t) = reshape(sol(t), (2n, 2n)); # Reformats vector into a density matrix

expx(t) = real(tr(solF(t)*kron(sx0, 𝕀b(n))));  # Sx expectation value
expy(t) = real(tr(solF(t)*kron(sy0, 𝕀b(n)))); # Sy expectation value
expz(t) = real(tr(solF(t)*kron(sz0, 𝕀b(n)))); # Sz expectation value

In [16]:
time = [i for i in t];
sx = [expx(i) for i in t];
sy = [expy(i) for i in t];
sz = [expz(i) for i in t];

In [17]:
df = DataFrame(hcat(time, sx, sy, sz), :auto);

In [20]:
CSV.write("dyn_data.csv", df, header = ["t", "sx", "sy", "sz"]); # Exports RC data to a CSV file

## Fidelity with MFGS

In [22]:
Htot1D = HRC1D(n, λ, Ω); # System Hamiltonian in original basis
P1D = eigen(Htot1D).vectors; # Find the projectors onto the system eigenbasis
Htot1DT = adjoint(P1D)*Htot1D*P1D; # System Hamiltonian in its eigenbasis
ρtot1D(T) = P1D*gibbs(Htot1DT, T)*adjoint(P1D); # Total Gibbs state in original basis
ρMFGS1D(T) = ptrace(ρtot1D(T), n); # Trace out the bath

In [33]:
time = [i for i in t];
fid = [real(ℱ(ptrace(solF(i), n), ρMFGS1D(T))) for i in t];
dfid = DataFrame(hcat(time, fid), :auto);

In [35]:
CSV.write("fid_data.csv", dfid, header = ["t", "fid"]); # Exports RC data to a CSV file